# Gemini Sentiment Analysis Agent Demo


### Get API keys

Before you can use the Gemini API, you must first obtain an API key. If you don't already have one, create a key with one click in [Google AI Studio](https://makersuite.google.com/app/apikey).  You'll also need an [OpenAI API key](https://openai.com/).

Once you have your API keys create the following Google Colab "secrets"
`GOOGLE_API_KEY` and `OPENAI_API_KEY` which will contain your respective keys.

## Initial Setup

In [1]:
!pip install llama-index google-generativeai openai pypdf -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.2/990.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.9/277.9 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 re

In [2]:
import os
from google.colab import userdata
import openai

openai.api_key = userdata.get('OPENAI_API_KEY')

In [3]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO, force=True)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

### Vectorize Mindfulness Guides

In [4]:
import os.path
from llama_index import (
  VectorStoreIndex,
  SimpleDirectoryReader,
  StorageContext,
  load_index_from_storage,
  )

# Attempt to load embeddings
try:
  storage_context = StorageContext.from_defaults(
      persist_dir="./storage/challenging_child"
  )
  challenging_child_index = load_index_from_storage(storage_context)

  storage_context = StorageContext.from_defaults(
      persist_dir="./storage/mindfulness_TB_50"
  )
  mindfulness_TB_50_index = load_index_from_storage(storage_context)

  storage_context = StorageContext.from_defaults(
      persist_dir="./storage/mindfulness_TB_relationships"
  )
  mindfulness_TB_relationships_index = load_index_from_storage(storage_context)
except:
  # Need to generate embeddings
  challenging_child_docs = SimpleDirectoryReader(
      input_files=["./data/The_Challenging_Child_Toolbox.pdf"]
  ).load_data(show_progress=True)
  mindfulness_TB_50_docs = SimpleDirectoryReader(
      input_files=["./data/The_Mindfulness_Toolbox__50_Practical_Tips_Tools.pdf"]
  ).load_data(show_progress=True)
  mindfulness_TB_relationships_docs = SimpleDirectoryReader(
      input_files=["./data/The_Mindfulness_Toolbox_for_Relationships.pdf"]
  ).load_data(show_progress=True)

  # Build index
  challenging_child_index = VectorStoreIndex.from_documents(challenging_child_docs)
  mindfulness_TB_50_index = VectorStoreIndex.from_documents(mindfulness_TB_50_docs)
  mindfulness_TB_relationships_index = VectorStoreIndex.from_documents(mindfulness_TB_relationships_docs)

  # Persist
  challenging_child_index.storage_context.persist(persist_dir="./storage/challenging_child")
  mindfulness_TB_50_index.storage_context.persist("./storage/mindfulness_TB_50")
  mindfulness_TB_relationships_index.storage_context.persist(persist_dir="./storage/mindfulness_TB_relationships")

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
NumExpr defaulting to 2 threads.


Loading files: 100%|██████████| 1/1 [00:16<00:00, 16.65s/file]
[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.open

In [5]:
challenging_child_engine = challenging_child_index.as_query_engine()
mindfulness_TB_50_engine = mindfulness_TB_50_index.as_query_engine()
mindfulness_TB_relationships_engine = mindfulness_TB_relationships_index.as_query_engine()

### Test to make sure our data loaded

In [6]:
from IPython.display import Markdown

In [7]:
# The_Challenging_Child_Toolbox
query_engine = challenging_child_index.as_query_engine()
response = query_engine.query("What is the definition of a challenging child?")
display(Markdown(f"**Response:**\n\n{response}"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**Response:**

A challenging child refers to a child who has emotional and behavioral difficulties. They may require additional support and intervention to address their challenges and help them meet new, higher standards for their actions in daily life.

In [8]:
# The_Mindfulness_Toolbox__50_Practical_Tips_Tools
query_engine = mindfulness_TB_50_index.as_query_engine()
response = query_engine.query("What are the benefits of a mindfulness practice?")
display(Markdown(f"**Response:**\n\n{response}"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**Response:**

Mindfulness practice has several benefits. It enhances flexibility and adaptability, allowing individuals to break free from old habits and retrain their brain. It cultivates curiosity and greater ease, encouraging exploration of the journey and process rather than being overly focused on the outcome. Mindfulness also changes one's relationship to self-critical and self-blaming thoughts, promoting greater patience, kindness, acceptance, and hospitality towards oneself and others. Additionally, mindfulness encourages greater fulfillment in daily life by focusing on the present moment, reducing rumination and negative thoughts, as well as anxiety about the future.

In [9]:
# The_Mindfulness_Toolbox_for_Relationships
query_engine = mindfulness_TB_relationships_index.as_query_engine()
response = query_engine.query("How does mindfulness relate to relationships?")
display(Markdown(f"**Response:**\n\n{response}"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**Response:**

Mindfulness relates to relationships by engaging a sense of curiosity and openness, which helps individuals escape the limited perspective of the ego. It allows individuals to experience others, even difficult people, in a new way. Mindfulness also helps individuals break free from old habits and toxic ways of reacting in relationships. By cultivating awareness, compassion, and openness, mindfulness provides practical tools for addressing issues that impact daily living and the stresses of the real world. Additionally, mindfulness can counteract the challenges posed by technology and social media, which can hinder face-to-face human connection and empathy. Overall, mindfulness skillfully uplifts and inspires individuals in their relationships with others.

## Define Agent Tools

### Analyze image tool

In [10]:
from llama_index.tools import BaseTool, FunctionTool
from llama_index.multi_modal_llms.gemini import GeminiMultiModal
from llama_index.multi_modal_llms.generic_utils import load_image_urls
from typing import List

In [11]:
def analyze_image(img_urls: List[str]) -> str:
  """Calls our Gemini vision API to analyze the image and return a description of the text contained in the image.
      Returns: A string with a description of the image and the mood it conveys if any.

  Args:
      img_urls (List[str]): The URL of one or more images that convey the users mood
  """
  image_documents = load_image_urls(img_urls)

  gemini = GeminiMultiModal(model="models/gemini-pro-vision", api_key=userdata.get('GOOGLE_API_KEY'))

  complete_response = gemini.complete(
      prompt="Identify what you see in the image and what mood it conveys if any",
      image_documents=image_documents,
  )

  return complete_response

vision_tool = FunctionTool.from_defaults(fn=analyze_image)

### Save session tool

In [ ]:
def save_session(chat_history: List[ChatMessage]) -> bool:
  """Persists the user's chat history. Use this tool when the user is happy with your recomendations and done with the session
      Returns: A boolean saying if the chat history was persisted

  Args:
      chat_history (List[ChatMessage]): The agent's chat history
  """

  # Here we'd ask the LLM to sumarize our chat session and then we'd persist it so we could restore it next time.

  return True

save_tool = FunctionTool.from_defaults(fn=save_session)

### Mindfulness routine recomendation tool

In [12]:
from llama_index.tools import QueryEngineTool, ToolMetadata

query_engine_tools = [
    QueryEngineTool(
        query_engine=challenging_child_engine,
        metadata=ToolMetadata(
            name="challenging_child",
            description=(
                "The Challenging Child Toolbox 75 Mindfulness Based Practices Tools and Tips for Therapists"
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=mindfulness_TB_50_engine,
        metadata=ToolMetadata(
            name="mindfulness_TB_50",
            description=(
                "The Mindfulness Toolbox 50 Practical Tips Tools Handouts for Anxiety Depression Stress and Pain"
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=mindfulness_TB_relationships_engine,
        metadata=ToolMetadata(
            name="mindfulness_TB_relationships",
            description=(
                "The Mindfulness Toolbox for Relationships 50 Practical Tips Tools Handouts for Building Compassionate Connections"
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

In [13]:
tools = query_engine_tools + [vision_tool]

## Create LlamaIndex Agent

### System prompt to give our agent some personality

In [14]:
SYSTEM_PROMPT = """You are an emotional support assistant with the expertise of an experienced counselor. Your primary role is to assist the user by encouraging them to provide a drawing that conveys their mood.
You offer professional, friendly, and helpful guidance based on current counseling and mindfulness practices. Once you receive the image, interpret it to discern what the user might be feeling and confirm with them if your observation is correct.
If your interpretation does not align with their feelings, engage in a dialogue until you accurately understand their mood. Your knowledge is exclusively focused on understanding the user's emotions and recommending mindfulness routines using the tool, tailored to their mood.
Thus, you will only provide responses related to these areas. If a question falls outside your area of expertise or if you lack the necessary information, you will inform the user by saying,
'Sorry, I do not know the answer to your question.' and then prompt for more information related to their feelings. Once they confirm that you have correctly understood their feelings, your task is to recommend a suitable mindfulness routine using the tool."""

### Init our agent

Using OpenAIAgent

In [61]:
from llama_index.agent import OpenAIAgent
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-4-1106-preview") # Using GPT-4 Turbo (Beta)

agent = OpenAIAgent.from_tools(
    tools,
    llm=llm,
    verbose=True,
    system_prompt=SYSTEM_PROMPT,
)

Alternative approach using ReActAgent so we can use Gemini as our LLM.  ReActAgent is designed to allow alternatives to OpenAI.

NOTE: This never worked reliably with Gemini because it's not conversational.  If you say 'Hi' it trys to ue a tool rather than just chatting.  Would be interesting to try it with other LLMs.  I tried using GPT-4 Turbo and it seemed to work.

In [16]:
# from llama_index.llms import Gemini, ChatMessage
# from llama_index.agent import AgentRunner, ReActAgentWorker, ReActAgent

# llm = Gemini(model="models/gemini-pro", api_key=userdata.get('GOOGLE_API_KEY'), temperature=0.1)
# agent = ReActAgent.from_tools(tools, llm=llm, verbose=True, chat_history=[ChatMessage(role="user", content=SYSTEM_PROMPT)])

## Test Our Agent

I'm picturing the frontend give the user the option to chat or attach one or more images that would get uploaded to cloud storage.  Then our back could prompt the agent with something like "Here's a drawing of how I'm feeling {Image URL Here}."

For the demo below you need to manually paste the image URL into the chat (i.e. https://ih1.redbubble.net/image.3636044620.1142/bg,f8f8f8-flat,750x,075,f-pad,750x1000,f8f8f8.u5.jpg)

Single prompt example

In [63]:
response = agent.chat("hi I'm Andrew")
display(Markdown(f"**Response:**\n\n{response}"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**Response:**

Hello again, Andrew! It's nice to hear from you. If you'd like to express how you're feeling through a drawing, please go ahead and share it with me. I'm here to help you interpret your emotions and provide support.

Multi-turn chat example

In [18]:
response = agent.chat_repl()

===== Entering Chat REPL =====
Type "exit" to exit.

Human: exit


## Reset Agent State

The agent's knowledge is based on it's chat history which we can easily reset.

In [32]:
agent.reset()
agent.chat_history

[]

If we ask it our name it shouldn't know

In [33]:
response = agent.chat("whats my name?")
display(Markdown(f"**Response:**\n\n{response}"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**Response:**

Sorry, I do not know the answer to your question. If you're feeling a certain way and would like to express it through a drawing, please share it with me, and I can help you explore your emotions and recommend a mindfulness routine.

Let's tell it our name and ask again

In [46]:
response = agent.chat("hi I'm Andrew")
display(Markdown(f"**Response:**\n\n{response}"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**Response:**

Hello Andrew! How can I assist you today? If you're feeling up to it, you can share a drawing that conveys your mood, and I'll do my best to understand and support you.

In [35]:
response = agent.chat("whats my name?")
display(Markdown(f"**Response:**\n\n{response}"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**Response:**

Your name is Andrew. If you're feeling a certain way and would like to express it through a drawing, please share it with me, and I can help you explore your emotions and recommend a mindfulness routine.

It knows our name and we can see the chat history

In [47]:
agent.chat_history

[ChatMessage(role=<MessageRole.USER: 'user'>, content="hi I'm Andrew", additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content="Hello Andrew! How can I assist you today? If you're feeling up to it, you can share a drawing that conveys your mood, and I'll do my best to understand and support you.", additional_kwargs={})]

As a sanity check lets reset the chat history one more time and ask it our name again.

In [51]:
agent.reset()
agent.chat_history

[]

In [38]:
response = agent.chat("whats my name?")
display(Markdown(f"**Response:**\n\n{response}"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**Response:**

Sorry, I do not know the answer to your question. Let's focus on how you're feeling today. If you'd like, you can share a drawing that conveys your mood, and I can help you interpret it and suggest a mindfulness routine that might be beneficial for you. Would you like to share a drawing with me?

## Handle Returning User

When returning to the conversation we can start with a chat history, even if the agent was reset, so the agent knows what we talked about last.

As a sanity check lets first create an agent without a chat history.

In [58]:
agent.reset()
agent.chat_history

[]

In [54]:
response = agent.chat("whats my name?")
display(Markdown(f"**Response:**\n\n{response}"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**Response:**

Your name is Andrew. How are you feeling today, Andrew? If you'd like, you can express your mood through a drawing and share it with me. I'm here to support you.

It doesn't know our name.  Now let's pass a chat history with our name into the chat message.  This is how we would handle multi-tenancy by tracking each user's conversation separantly and passing it into the chat message to the agent.

In [59]:
from llama_index.llms import ChatMessage

chat_history = [ChatMessage(role="user", content="hi I'm Andrew", additional_kwargs={})]

In [60]:
response = agent.chat("whats my name?", chat_history=chat_history)
display(Markdown(f"**Response:**\n\n{response}"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**Response:**

Hello Andrew! How can I assist you today? Would you like to share a drawing that conveys your mood?